# 트립 어드바이저 데이터 수집



In [1]:
from selenium import webdriver
import time
from pygrok import Grok
import pandas as pd
from ast import literal_eval 
from tqdm import notebook  # for문 진행상황 확인

In [2]:
# 초기값
data_list = []
error_list = []

driver = webdriver.Chrome("./chromedriver")  

In [3]:
def get_rating(rating_html):    
    if "bubble_50" in rating_html:
        rating = 5
    elif "bubble_40" in rating_html:
        rating = 4
    elif "bubble_30" in rating_html:
        rating = 3
    elif "bubble_20" in rating_html:
        rating = 2
    elif "bubble_10" in rating_html:
        rating = 1
    return rating

def get_date(date_text):
    
    print(date_text)
    date_pattern = '%{YEAR:year}년 %{MONTHNUM:month}월'
    pattern = Grok(date_pattern)
    grok = pattern.match(date_text)

    if grok is None:
        date_pattern = '%{MONTHNUM:month}월'
        pattern = Grok(date_pattern)
        grok = pattern.match(date_text)        
        grok['year'] = '2021'

    date = grok['year'] + "/" + grok['month']
    return date


def get_data(page, url, place, category, driver):

    # 사용자 id 수집
    users = driver.find_elements_by_css_selector('._1r_My98y')  

    # 별점 수집
    ratings = driver.find_elements_by_css_selector('.nf9vGX55')
    
    # 리뷰 수집
    review_titles = driver.find_elements_by_css_selector('.ocfR3SKN')
    review_descs = driver.find_elements_by_css_selector('.IRsGHoPm')
    
    # 날짜 수집
    dates = driver.find_elements_by_css_selector('._2fxQ4TOx')
    
    is_normal = False
    
    if len(users) == len(ratings) == len(review_titles) == len(review_descs) == len(dates):
            is_normal = True
    
    if is_normal:
        
        for index in range(len(users)):
                  
            user_link = users[index].get_attribute('href')        
            user_id = user_link.rsplit('/', 1)[-1]
            
            rating_score = get_rating(ratings[index].get_attribute('innerHTML'))
            
            review = review_titles[index].text + "\n" + review_descs[index].text
            
            date = get_date(dates[index].text)
            
            tag = []
            tag.append(category)
            
            data = {}
            data['user_id'] = user_id
            data['rating'] = rating_score
            data['review'] = review
            data['date'] = date
            data['place'] = place
            data['url'] = url
            data['tag'] = tag
            
            print(data)
            data_list.append(data)
    
    else:
        print("수집 중 오류 발생 user id, ratings, review, dates 개수가 서로 다름")
        
        error_data = {}
        error_data['place'] = place
        error_data['url'] = url
        error_data['review_page'] = page
        
        print("error_data: " + error_data)
        error_list.append(error_data)


In [4]:
def get_data_from_url(url, place, category, driver):
    
    driver.get(url)
    time.sleep(3)
    paginations = driver.find_elements_by_css_selector('.ui_pagination a')
    
    if len(paginations) > 1:
        last_page = int(paginations[-1].get_attribute('innerHTML'))
    else:
        last_page = 1
    print('last_page ' + str(last_page))
    
    page = 1
    print("page 1")
    get_data(page, url, place, category, driver) # 첫 페이지 수집하고 시작

    for page in range(2, last_page + 1):
        print("page " + str(page))
        next_button = driver.find_element_by_css_selector('.ui_button.nav.next.primary')
        next_button.click()
        time.sleep(3)
        get_data(page, url, place, category, driver)

In [10]:
# 총 2487

len(urls)

2487

In [5]:
# 파일 읽어오기
places_data = pd.read_excel('./places.xlsx')
categories = places_data['category']
places = places_data['place']
urls = places_data['url']

# range(시작, 끝) 예) range(1000, 1500)
for index in notebook.tqdm(range(823, 825)):
    url = urls[index]
    place = places[index]
    print()
    print(place)
    category = categories[index]
    get_data_from_url(url, place, category, driver)


서울이스케이프룸 - 강남1호점
last_page 83
page 1
Dongho K이 리뷰를 작성했습니다. 1월 2일
{'user_id': 'donghok2021', 'rating': 5, 'review': '깔끔한 설명과 플레이\n재밌었어요#~~~신나는 하루됐습니다~~~~~~배경성명부터 흥미진진했어여~~~~~~ 아마존의 잃어버린도시 베니스상인의 저주받은 저태구추천합니당~~~저희는 응답하라 1988했어요 너무재밌더', 'date': '2021/1', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
Jaem이 리뷰를 작성했습니다. 1월 2일
{'user_id': 'X5347IO_', 'rating': 4, 'review': 'Play\n응답하아1988 개꿀잼이에여~!!!! 서울이스케이프룸에서 횡단열차랑 죽음의 재즈바 했었는데 응답하라1988도 너무 재밋었어요ㅎㅎㅎㅎ강남에서는 무조건 서이룸 강추해요-;!!! 재방문할게요~', 'date': '2021/1', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
S H S이 리뷰를 작성했습니다. 2020년 12월
{'user_id': 'SHS0322', 'rating': 5, 'review': 'good experience\n탈출하라1988 진행했습니다. 처음 주어진 공간에서 차례차례 단서를 찾아가다보면 새로운 공간으로 이동하게 되는데 이 공간이 여러번 반복되어 지루하지 않고 흥미 있었습니다. 

YoungCheol이 리뷰를 작성했습니다. 2020년 10월
{'user_id': '5dmark4', 'rating': 5, 'review': '알카트라즈 지하감옥 후기\n생애 첫 방탈출을 경험했던 서이룸 1호점이였는데 정말 오랜만에 재방문했다! 오랜만에 했더니 생각외로 정말 재미있었고 왠만한 매장보다 퀄리티도 높은 편이였다 아직 몇가지 해보지 않았는데 다른 테마들도 더 하고싶게 만들어주는 테마였더', 'date': '2020/10', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
Soomin C이 리뷰를 작성했습니다. 2020년 10월
{'user_id': 'soominc2020', 'rating': 5, 'review': 'veryfun\n서울이스케이프룸 정말 재미있습니다! 탈출하라 1988 완전 꿀잼입니다! 인테리어도 리얼하고 문제 푸는 재미도 있습니다. 유럽횡단 야간열차도 완전 꿀잼입니다 꼭 해보세요! 화성탐사선 언제 나오는지 궁금 ㅠㅠ', 'date': '2020/10', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
두앵 서이 리뷰를 작성했습니다. 2020년 9월
{'user_id': 'D6896KS_', 'rating': 5, 'review': '존잼\n퀄리티쩌러요~ 너무 재밋졍!!!! 다들 벌써 7번쨔 도전중 ㅠㅠ 다들 꼭 방문해서 좋은추억 남기세요. 다들 시즌별로 이야기 이어서 꼭 진행하세요gooooood', 'd

qwerty이 리뷰를 작성했습니다. 2020년 8월
{'user_id': 'qwerty1010101010', 'rating': 5, 'review': '재밌었어요!\n친하지 않은 친구들과 친해질 수 있었어요 코로나와 맞지않은 부분이 조금 있긴한데.. 아무튼 실내에서 여럿이서 즐기기 좋아요', 'date': '2020/8', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
이승현이 리뷰를 작성했습니다. 2020년 8월
{'user_id': 'Fabian805', 'rating': 5, 'review': '방탈출카페 추천!\n친구들과 놀러와서 오랜만에 재밌게 놀았습니다! 다른 곳에 비해 다양한 테마가 있고, 방 구성이 되게 새로웠어요! 다른 분들에게도 적극 추천합니다~', 'date': '2020/8', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
HeeKIM이 리뷰를 작성했습니다. 2020년 8월
{'user_id': 'HeeKIM', 'rating': 5, 'review': 'Really good\n죽재바. 예상치못한 ! 그런방입니다 요새는워낙 크기가압도적인 방탈이 많은데 여기는 다른 매력포인트가있어욧 가격은좀 쎄지만 그만큼 이유가있습니다.', 'date': '2020/8', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g2

최 승이 리뷰를 작성했습니다. 2020년 7월
{'user_id': 'J4944BJ_', 'rating': 5, 'review': 'Good\n유럽 횡단 야간 열차 도전해봤어요! 열차라는 설정에 걸맞게 열차가 실제로 움직여서 재밌고 신기했어요 플랫폼이라던가 내부 구조도 그렇고 설정에 잘 맞춰서 잘 구현해둬서 더 즐길 수 있었습니다 스토리도 탄탄하고 대화카드를 통해 스토리속 인물과 대화하듯 진행할 수 있어서 더 몰입할 수 있었던 것 같아요 다음에 또 와서 다른 것도 플레이해보고싶어요! おすすめします💞 Good', 'date': '2020/7', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
ㅇㅇㅇ이 리뷰를 작성했습니다. 2020년 7월
{'user_id': 'E6018BG_', 'rating': 5, 'review': '유럽횡단야간열차 테마\n유럽횡단야간열차 테마 플레이했어요 다른테마보다 가격은 비싼데 대신 정말 열차가 움직여서 현실감이 높아서 재밌었어요! 마지막에 범인 맞추는것까지 해야하는데 좀 어렵더라고요ㅠ 다른 테마도 다음에 해보고싶어요!good', 'date': '2020/7', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
hoyomi이 리뷰를 작성했습니다. 2020년 7월
{'user_id': 'hoyomi', 'rating': 5, 'review': 'ㅈ ㅐ미있어요\n밥먹고 와서 소화 시키기 좋아요,,, 편한 옷 입고오

너에게가는길이 리뷰를 작성했습니다. 2020년 5월
{'user_id': 'Way2u', 'rating': 5, 'review': '서이룸 유럽횡단야간열차 재밌네요!\n단서가 많고 스토리도 탄탄해서 재밌네요 서이룸 스타일 맘에듭니다! 장치가 너무 신기했어요ㅎㅎ 문제퀄도 좋았습니다!!!ㅎㅎ', 'date': '2020/5', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
채영심이 리뷰를 작성했습니다. 2020년 5월
{'user_id': 'Nomad810416', 'rating': 5, 'review': 'Seoul escape room\n유럽횡단야간열차했습니다~ 소문대로 연출이 너무 좋네요!! 재밌게 즐겼습니다! 시즌1,2,3 다해가는 중인데 거의다 마음에 들어요! 빨리 화상탐사선 열어서 가보고싶네요 ㅠㅠ 추천합니다!', 'date': '2020/5', 'place': '서울이스케이프룸 - 강남1호점', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d10725552-Reviews-Seoul_Escape_Room_GangNam_1-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
박용민이 리뷰를 작성했습니다. 2020년 5월
{'user_id': 'Youngmi2', 'rating': 5, 'review': '방탈~~출!!!\n광주에서 서울까지 방탈출을 뿌셔뿌셔 하러 왔습니다~ 와 알카트라즈감옥 노힌트로 13분 가량 남기고 미션 완료ㅎㅎㅎ 시간이 남아 바로 야간열차를 도전하였지만 어이쿠 너무 어려워서 힌트 남발 하였네요 그래도 즐거운시간 보내기가 좋았습니다 다음에도 서울 올일 있다면 다른 시즌 참가해 봐야겠어용

...

solarccc이 리뷰를 작성했습니다. 2020년 1월
{'user_id': 'solarccc', 'rating': 5, 'review': '셜록\n즐겁다... 방 스케일이 좀 더 커도 좋았을 것 같아요 목소리 큰 사람과 있으면 귀가 아파요 ... 소통이 잘 되는 친구와 가지 않으면 손절할 수 있습니다 네명정도면 좋을 것 같아요', 'date': '2020/1', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
hyeseong Kim이 리뷰를 작성했습니다. 2020년 1월
{'user_id': 'athon_k', 'rating': 5, 'review': '아르바이트\n어우 너무 좋았어요!! 어려운데 탈출성공! 조도 많이 어두워서 힘들었지만 그래도 재밌어서 크게 신경쓰이진 않았네요~ 다음에 또 올께요!', 'date': '2020/1', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
kimpzipu이 리뷰를 작성했습니다. 2020년 1월
{'user_id': 'ldh852', 'rating': 5, 'review': '개인적으론 상당히 어려웠어요\n연출은 간결한 편인데 뭐랄까 문제 간에 헷갈릴수 있는 그런게 많았네요 저흰 힌트도 다쓰고 아슬아슬하게 클리어했습니다 ㅠㅠ 난해한 벽에 부딪히는 느낌 좋아하신다면 강추', 'date': '2020/1', 'place': '리얼 이스케이프 체

sungeun lee이 리뷰를 작성했습니다. 2019년 12월
{'user_id': 'dars221', 'rating': 5, 'review': '방탈출 처음인데 또하고싶어요!!\n너무 재밌었어요~~ 탈출 못해서 너무 아쉬운데 다음에 쉬운걸로 다시 도전하고 싶어요ㅠㅠ 홍대 리얼이스케이프 강추👍👍', 'date': '2019/12', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
Hansol이 리뷰를 작성했습니다. 2019년 12월
{'user_id': 'gusul_handol', 'rating': 5, 'review': '셜록홈즈 테마 방탈출하기!\n처음 방탈출 해보는 거 였는데 너무 재미있었어요! 무서운거 안하려다보니 셜록을 선택하게 되었는데 다음 번에는 조금 쉬운걸로 친구들이랑 다시 해보고싶어요! 친구들과 좋은 추억 남길 수 있어 좋았습니당~', 'date': '2019/12', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
Jisoo J이 리뷰를 작성했습니다. 2019년 12월
{'user_id': 'JisooReese', 'rating': 5, 'review': '친구들끼리 첫도전\n첫도전에 최고난이도 셜록했다가 갇혔어요 ㅋㅋㅋㅋㅋㅋ 그래도 또 오고 싶을만큼 재밌었어요 :) 담엔 힌트 없이 탈출할래요.. ㅜㅜ ㅋㅋㅋㅋ', 'date': '2019/12', 'place': '리얼 이

수진 박이 리뷰를 작성했습니다. 2019년 11월
{'user_id': '_I530FQ', 'rating': 5, 'review': '방탈출재미있어요ㅋㅋ\n친구들이랑 저녁먹고나서 바로 방문해서 구미호 테마했는데재밌었어요ㅋ 추리하느라 시간가는줄 몰랐네요 다들 탈출하세요!!', 'date': '2019/11', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
민아 강이 리뷰를 작성했습니다. 2019년 11월
{'user_id': 'minature', 'rating': 4, 'review': '리얼 이스케이프 구미호테마\n구미호테마 무서울 줄 알고 겁먹었는데 재밌게 했어요. 저희는 실패했는데 어차피 못풀거라 아쉽지는 않았습니다. 방탈출 추천 ㅎㅎ', 'date': '2019/11', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
임설희이 리뷰를 작성했습니다. 2019년 11월
{'user_id': 'ooohoi', 'rating': 5, 'review': '신나는 방탈출~\n구미호 테마 신선한 스토리 예상치 못한 퀴즈로 탈출하는 재미가 있었습니다. 다음에 다른테마도 해보고싶어요.. 무서운 소품들이 많아서 좋았고, 탈출퀴즈들도 신선했습니다', 'date': '2019/11', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co

SU237이 리뷰를 작성했습니다. 2016년 1월
{'user_id': 'SU237', 'rating': 4, 'review': '방탈출게임!\n친구들이나 커플끼리 가기 좋음. 방안에 있는 힌트들을 이용해서 방을 탈출해야 하는데 꽤 어려웠지만 충분히 탈출할 수 있을 정도의 난이도임! 주의할 점은 미리 예약을 해야하는데 방탈출카페가 서울에 많이 있는게 아니라 며칠 전부터 예약하지 않으면 가기 힘들다.', 'date': '2016/1', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
Sasha이 리뷰를 작성했습니다. 2019년 9월
{'user_id': 'Sashakim1004', 'rating': 5, 'review': '즐거운 시간을 보냈습니다!\n우리는 여기서 즐거운 시간을 보냈습니다. 탈출 실 게임을 많이하고 여기 추천합니다. 한국과 영어를 모두 갖춘 훌륭한 객실', 'date': '2019/9', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
kkholiday231이 리뷰를 작성했습니다. 2019년 9월
{'user_id': 'kkholiday231', 'rating': 5, 'review': '셜록 룸\n좋은 퍼즐. 방에 좋은 흐름. 직원들은 친절했고 단서도 빨리주었습니다. 전체 경험을 철저히 즐겼습니다.', 'date': '2019/9', 'place': '리얼 이스케이프

Wanderer796267이 리뷰를 작성했습니다. 2018년 10월
{'user_id': 'Wanderer796267', 'rating': 5, 'review': '좋은 경험\n방은 많은 즐거움뿐만 아니라 상당히 도전적이었습니다! 개인은 큰 영어를 말했고, 모든 것을 매우 잘 설명했다! 훌륭한 가격과 큰 재미!', 'date': '2018/10', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
Julia W이 리뷰를 작성했습니다. 2018년 10월
{'user_id': 'P2755QIjuliaw', 'rating': 5, 'review': '멋진 재미!\n우리는 6 명으로 나뉘 었습니다. 우리 대부분은 탈출실에 처음갔습니다. 우리는 sherlock 도전을 골랐다. 그리고 그것은 컸다!', 'date': '2018/10', 'place': '리얼 이스케이프 첼린지', 'url': 'https://www.tripadvisor.co.kr/Attraction_Review-g294197-d9715906-Reviews-Real_Escape_Challenge_Hongdae-Seoul.html', 'tag': ['서울 액티비티 / 게임']}
velocitychee이 리뷰를 작성했습니다. 2018년 7월
{'user_id': 'velocitychee', 'rating': 5, 'review': '재밌.\n정말 재밌었어요, 7 좋은 도전이(우리는 셜록 그리고 1분 그쳤습니다. 직원들은 영어를 단서를 모두 영어를 잘 했습니다. 꼭 다시! 팀워크 많이 필요 하지만 너무 작은 그룹에 대한 것이다.', 'date': '2018/7', 'place': '리얼 이스케이ᄑ

In [6]:
print(len(data_list))
print(len(error_list))

628
0


In [7]:
data_table = pd.DataFrame(data_list)
data_table      


,user_id,rating,review,date,place,url,tag
0,donghok2021,5,깔끔한 설명과 플레이\n재밌었어요#~~~신나는 하루됐습니다~~~~~~배경성명부터 흥...,2021/1,서울이스케이프룸 - 강남1호점,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
1,X5347IO_,4,Play\n응답하아1988 개꿀잼이에여~!!!! 서울이스케이프룸에서 횡단열차랑 죽음...,2021/1,서울이스케이프룸 - 강남1호점,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
2,SHS0322,5,good experience\n탈출하라1988 진행했습니다. 처음 주어진 공간에서 ...,2020/12,서울이스케이프룸 - 강남1호점,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
3,chan021226,5,진짜재밌어요\n죽음을 부르는 재즈바를 다녀왔었는데 너무 신박하고 새로웠습니다. 분위...,2020/12,서울이스케이프룸 - 강남1호점,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
4,Peleles,5,완성도 높은 서울 대표 방탈출\n유럽횡단열차 완성도 높고 재밌었어요! 다른 방탈출 ...,2020/11,서울이스케이프룸 - 강남1호점,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
...,...,...,...,...,...,...,...
623,raechelw2018,5,흥미롭고 매우 재미!\n내 다른 미국 친구들과 처음으로 갔는데 모두 즐거운 시간을 ...,2018/4,리얼 이스케이프 첼린지,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
624,jolenek547,5,정말 흥미로운!\n이 이스케이프 방을 처음 매우 흥미로운! 이 객실은 한국어와 영어...,2018/4,리얼 이스케이프 첼린지,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
625,SaesaemO1,5,"홍대에 즐거운 경험!!\n남자친구와 함께 갔는데, Cupid 테마. 나는 객실은 여...",2018/3,리얼 이스케이프 첼린지,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]
626,pinkkey90,5,처음에 방을 벗어나!\n5 중 어떤 유형의 방은 아주 흥미로운 탈출 할 수 있는! ...,2017/7,리얼 이스케이프 첼린지,https://www.tripadvisor.co.kr/Attraction_Revie...,[서울 액티비티 / 게임]


In [8]:
error_table = pd.DataFrame(error_list)
error_table

""


In [9]:
# 엑셀로 저장하기
data_table.to_excel("tripadvisor.xlsx")